In [1]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

In [2]:
os.makedirs("images8", exist_ok=True)
n_epochs = 200
batch_size = 64
lr_g = 0.0001
lr_d = 0.0004
b1 = 0.5
b2 = 0.999
latent_dim = 128
channels = 3
n_critic = 5
sample_interval = 400
img_shape = (3, 56, 56)
img_size = 28
cuda = True if torch.cuda.is_available() else False

In [3]:
data_transform = transforms.Compose([
                               transforms.CenterCrop(138), 
                               transforms.Resize(56),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
training_data = datasets.ImageFolder(root='.\data',transform=data_transform)
print(training_data)
dataloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4)


Dataset ImageFolder
    Number of datapoints: 202599
    Root location: .\data
    StandardTransform
Transform: Compose(
               CenterCrop(size=(138, 138))
               Resize(size=56, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class Reshape1(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), 64, 7, 7)

def wasserstein_loss(y_true, y_pred):
    return torch.mean(y_true * y_pred)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()


        self.model = nn.Sequential(
              
            nn.Linear(latent_dim, 3136, bias=False),
            nn.BatchNorm1d(num_features=3136),
            nn.LeakyReLU(inplace=True, negative_slope=0.0001),
            Reshape1(),
            
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=(3, 3), stride=(2, 2), padding=1, bias=False),
            nn.BatchNorm2d(num_features=32),
            nn.LeakyReLU(inplace=True, negative_slope=0.0001),
            #nn.Dropout2d(p=0.2),
            
            nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=(3, 3), stride=(2, 2), padding=1, bias=False),
            nn.BatchNorm2d(num_features=16),
            nn.LeakyReLU(inplace=True, negative_slope=0.0001),
            #nn.Dropout2d(p=0.2),
            
            nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=0, bias=False),
            nn.BatchNorm2d(num_features=8),
            nn.LeakyReLU(inplace=True, negative_slope=0.0001),
            #nn.Dropout2d(p=0.2),
            
            nn.ConvTranspose2d(in_channels=8, out_channels=3, kernel_size=(2, 2), stride=(1, 1), padding=0, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, padding=1, kernel_size=(3, 3), stride=(2, 2), bias=False),
            nn.BatchNorm2d(num_features=8),
            nn.LeakyReLU(inplace=True, negative_slope=0.0001), 
            #nn.Dropout2d(p=0.2),
            
            nn.Conv2d(in_channels=8, out_channels=16, padding=1, kernel_size=(3, 3), stride=(2, 2), bias=False),
            nn.BatchNorm2d(num_features=16),
            nn.LeakyReLU(inplace=True, negative_slope=0.0001), 
            #nn.Dropout2d(p=0.2),
            
            nn.Conv2d(in_channels=16, out_channels=32, padding=1, kernel_size=(3, 3), stride=(2, 2), bias=False),
            nn.BatchNorm2d(num_features=32),
            nn.LeakyReLU(inplace=True, negative_slope=0.0001), 
            #nn.Dropout2d(p=0.2),
            
            Flatten(),

            
            #nn.Sigmoid()
            )
        self.Li = nn.Linear(512, 1)

    def forward(self, img):
        validity = self.model(img)
        #print(validity.shape)
        validity = self.Li(validity)
        #print(validity.shape)
        return validity
    
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

print(generator)
print(discriminator)

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

print(generator)
print(discriminator)

In [4]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity


# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

if cuda:
    generator.cuda()
    discriminator.cuda()
print(generator)
print(discriminator)

Generator(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=9408, bias=True)
    (12): Tanh()
  )
)
Discriminator(
  (model): Sequential(
    (0): Linear(in_features=9408, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=Tr

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.ConvTranspose2d(100, 512, kernel_size = (4,4), stride=(1,1), bias=False),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size = (4,4), stride=(2,2), padding=(1,1), bias=False),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size = (4,4), stride=(2,2), padding=(1,1), bias=False),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 64, kernel_size = (4,4), stride=(2,2), padding=(1,1), bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity
    
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
print(generator)
print(discriminator)

In [6]:
# Optimizers
optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=lr_g)
optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=lr_d)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [6]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr_g, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr_d, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


In [7]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty
def adjust_learning_rate(optimizer, epoch, lr_in):
    """Sets the learning rate to the initial LR decayed by 10 every 15 epochs"""
    lr_ =  lr_in * (0.1 ** (epoch // 15))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_  

In [ ]:
# ----------
#  Training
# ----------
best_loss = 100.
batches_done = 0
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        generator.train()
        discriminator.train()
        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.uniform(0, 1, (imgs.shape[0], latent_dim))))
        #z = Variable(torch.randn(batch_size, latent_dim, 1, 1))
        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)

        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        adjust_learning_rate(optimizer_D, epoch, lr_d)
        adjust_learning_rate(optimizer_G, epoch, lr_g)
        # Train the generator every n_critic steps
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()
            
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )
            if abs(g_loss.item()) < best_loss:
                torch.save(generator,'./RMSprop_wgan')
                best_loss = abs(g_loss.item())
                print('model save at loss = %f' % (g_loss.item()))
            if batches_done % sample_interval == 0:
                generator.eval()
                save_image(fake_imgs.data[:9], "images8/%d.png" % batches_done, nrow=3, normalize=True)
                
            batches_done += n_critic

[Epoch 0/200] [Batch 0/3166] [D loss: 9.682660] [G loss: 0.033586]


C:\Users\Yi Feng\Anaconda3\envs\dl\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Generator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


model save at loss = 0.033586
[Epoch 0/200] [Batch 5/3166] [D loss: -23.463995] [G loss: 0.185594]
[Epoch 0/200] [Batch 10/3166] [D loss: -46.913036] [G loss: 0.475544]
[Epoch 0/200] [Batch 15/3166] [D loss: -46.136894] [G loss: 1.685362]
[Epoch 0/200] [Batch 20/3166] [D loss: -59.334358] [G loss: 0.921033]
[Epoch 0/200] [Batch 25/3166] [D loss: -69.302505] [G loss: 0.034190]
[Epoch 0/200] [Batch 30/3166] [D loss: -71.131973] [G loss: -0.331753]
[Epoch 0/200] [Batch 35/3166] [D loss: -65.500313] [G loss: -1.034641]
[Epoch 0/200] [Batch 40/3166] [D loss: -69.330200] [G loss: -1.129175]
[Epoch 0/200] [Batch 45/3166] [D loss: -66.217155] [G loss: -1.252615]
[Epoch 0/200] [Batch 50/3166] [D loss: -71.282394] [G loss: -1.433748]
[Epoch 0/200] [Batch 55/3166] [D loss: -72.225113] [G loss: -1.811457]
[Epoch 0/200] [Batch 60/3166] [D loss: -83.373154] [G loss: -2.133715]
[Epoch 0/200] [Batch 65/3166] [D loss: -86.545197] [G loss: -2.927700]
[Epoch 0/200] [Batch 70/3166] [D loss: -90.849213] [G

[Epoch 0/200] [Batch 570/3166] [D loss: -36.371948] [G loss: -5.762958]
[Epoch 0/200] [Batch 575/3166] [D loss: -42.340996] [G loss: -11.487638]
[Epoch 0/200] [Batch 580/3166] [D loss: -35.311115] [G loss: -4.973709]
[Epoch 0/200] [Batch 585/3166] [D loss: -37.795242] [G loss: -0.882629]
[Epoch 0/200] [Batch 590/3166] [D loss: -38.565647] [G loss: -3.332385]
[Epoch 0/200] [Batch 595/3166] [D loss: -37.719543] [G loss: -5.464477]
[Epoch 0/200] [Batch 600/3166] [D loss: -45.377098] [G loss: 6.514403]
[Epoch 0/200] [Batch 605/3166] [D loss: -43.323120] [G loss: -7.609234]
[Epoch 0/200] [Batch 610/3166] [D loss: -45.244095] [G loss: -1.100263]
[Epoch 0/200] [Batch 615/3166] [D loss: -41.419701] [G loss: -3.838773]
[Epoch 0/200] [Batch 620/3166] [D loss: -40.031860] [G loss: -1.489069]
[Epoch 0/200] [Batch 625/3166] [D loss: -46.141190] [G loss: -3.988760]
[Epoch 0/200] [Batch 630/3166] [D loss: -32.431259] [G loss: 1.157565]
[Epoch 0/200] [Batch 635/3166] [D loss: -45.424957] [G loss: 4.02

[Epoch 0/200] [Batch 1135/3166] [D loss: -34.290077] [G loss: 5.459374]
[Epoch 0/200] [Batch 1140/3166] [D loss: -29.536728] [G loss: 0.569596]
[Epoch 0/200] [Batch 1145/3166] [D loss: -29.400902] [G loss: -0.755195]
[Epoch 0/200] [Batch 1150/3166] [D loss: -31.591043] [G loss: 12.129928]
[Epoch 0/200] [Batch 1155/3166] [D loss: -32.190361] [G loss: 9.707707]
[Epoch 0/200] [Batch 1160/3166] [D loss: -29.899092] [G loss: -3.031967]
[Epoch 0/200] [Batch 1165/3166] [D loss: -30.087132] [G loss: 1.472071]
[Epoch 0/200] [Batch 1170/3166] [D loss: -30.199638] [G loss: -0.379412]
[Epoch 0/200] [Batch 1175/3166] [D loss: -30.595467] [G loss: 3.197158]
[Epoch 0/200] [Batch 1180/3166] [D loss: -27.538946] [G loss: -4.213981]
[Epoch 0/200] [Batch 1185/3166] [D loss: -30.657303] [G loss: 0.577175]
[Epoch 0/200] [Batch 1190/3166] [D loss: -33.293884] [G loss: 4.759147]
[Epoch 0/200] [Batch 1195/3166] [D loss: -33.838108] [G loss: -1.891172]
[Epoch 0/200] [Batch 1200/3166] [D loss: -29.614784] [G lo

[Epoch 0/200] [Batch 1700/3166] [D loss: -27.491570] [G loss: 5.687381]
[Epoch 0/200] [Batch 1705/3166] [D loss: -26.989864] [G loss: 5.662131]
[Epoch 0/200] [Batch 1710/3166] [D loss: -29.048790] [G loss: 0.654930]
[Epoch 0/200] [Batch 1715/3166] [D loss: -30.098579] [G loss: 6.492284]
[Epoch 0/200] [Batch 1720/3166] [D loss: -28.382822] [G loss: -2.826240]
[Epoch 0/200] [Batch 1725/3166] [D loss: -33.570965] [G loss: 4.664761]
[Epoch 0/200] [Batch 1730/3166] [D loss: -27.397285] [G loss: 9.017362]
[Epoch 0/200] [Batch 1735/3166] [D loss: -30.772480] [G loss: 15.295927]
[Epoch 0/200] [Batch 1740/3166] [D loss: -32.140381] [G loss: 6.441147]
[Epoch 0/200] [Batch 1745/3166] [D loss: -26.533167] [G loss: 7.509861]
[Epoch 0/200] [Batch 1750/3166] [D loss: -27.550083] [G loss: 0.284115]
[Epoch 0/200] [Batch 1755/3166] [D loss: -30.459370] [G loss: 7.343574]
[Epoch 0/200] [Batch 1760/3166] [D loss: -24.185223] [G loss: 11.227110]
[Epoch 0/200] [Batch 1765/3166] [D loss: -28.647287] [G loss:

[Epoch 0/200] [Batch 2265/3166] [D loss: -29.634262] [G loss: 5.983769]
[Epoch 0/200] [Batch 2270/3166] [D loss: -26.926226] [G loss: 5.779732]
[Epoch 0/200] [Batch 2275/3166] [D loss: -27.621412] [G loss: -1.880738]
[Epoch 0/200] [Batch 2280/3166] [D loss: -28.666771] [G loss: 8.629138]
[Epoch 0/200] [Batch 2285/3166] [D loss: -29.955847] [G loss: 13.313465]
[Epoch 0/200] [Batch 2290/3166] [D loss: -25.677214] [G loss: 7.328914]
[Epoch 0/200] [Batch 2295/3166] [D loss: -27.444889] [G loss: 9.664263]
[Epoch 0/200] [Batch 2300/3166] [D loss: -24.415693] [G loss: -0.272440]
[Epoch 0/200] [Batch 2305/3166] [D loss: -29.637550] [G loss: 3.931837]
[Epoch 0/200] [Batch 2310/3166] [D loss: -28.816452] [G loss: 11.727690]
[Epoch 0/200] [Batch 2315/3166] [D loss: -29.394266] [G loss: 2.092237]
[Epoch 0/200] [Batch 2320/3166] [D loss: -28.275894] [G loss: 4.304096]
[Epoch 0/200] [Batch 2325/3166] [D loss: -26.113033] [G loss: 10.657660]
[Epoch 0/200] [Batch 2330/3166] [D loss: -29.105040] [G los

[Epoch 0/200] [Batch 2830/3166] [D loss: -23.666855] [G loss: 4.573415]
[Epoch 0/200] [Batch 2835/3166] [D loss: -25.311363] [G loss: 1.727398]
[Epoch 0/200] [Batch 2840/3166] [D loss: -25.554684] [G loss: 6.559539]
[Epoch 0/200] [Batch 2845/3166] [D loss: -27.171156] [G loss: 0.753862]
[Epoch 0/200] [Batch 2850/3166] [D loss: -24.895824] [G loss: 3.494643]
[Epoch 0/200] [Batch 2855/3166] [D loss: -25.830433] [G loss: 8.722315]
[Epoch 0/200] [Batch 2860/3166] [D loss: -27.626846] [G loss: 6.069223]
[Epoch 0/200] [Batch 2865/3166] [D loss: -30.222885] [G loss: 10.100498]
[Epoch 0/200] [Batch 2870/3166] [D loss: -24.597725] [G loss: 2.273001]
[Epoch 0/200] [Batch 2875/3166] [D loss: -26.047409] [G loss: -1.760480]
[Epoch 0/200] [Batch 2880/3166] [D loss: -27.757332] [G loss: 5.775844]
[Epoch 0/200] [Batch 2885/3166] [D loss: -24.726669] [G loss: 11.920142]
[Epoch 0/200] [Batch 2890/3166] [D loss: -25.501385] [G loss: 7.420437]
[Epoch 0/200] [Batch 2895/3166] [D loss: -25.760477] [G loss:

In [1]:
for i in range(500):
    generator.eval()
    qq = Variable(Tensor(np.random.normal(0, 1, (9, latent_dim))))
    save = generator(qq)
    save_image(save.data[:9], "done_3/%03d.png" % i, nrow=3, normalize=True)

NameError: name 'generator' is not defined

In [23]:
torch.save(generator,'./RMSprop_wgan')

RuntimeError: cuda runtime error (30) : unknown error at C:\w\1\s\tmp_conda_3.6_045031\conda\conda-bld\pytorch_1565412750030\work\torch/csrc/generic/serialization.cpp:23